# Notebook to Segment Skeletal Muscle Area from MRI images (TIFs)

###### Load libraries and directories

In [1]:
import os
os.getcwd()
os.chdir('/tf/smipipeline')
print(os.getcwd())

/tf/smipipeline


In [2]:
from IPython.display import display, HTML

In [3]:
# from IPython import get_ipython
from tqdm.notebook import tqdm
import pickle
import os
import pprint


import tables
from MRI_notebooks.TIF_loader import mri_TIF_loader
from unet3d.normalize import normalize_data_storage_2D


pp = pprint.PrettyPrinter(indent=1)
 
import json
import pprint

# Custom functions
import pickle
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

def load_object(filename):        
    with open(filename, 'rb') as input:
        return pickle.load(input)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
Using TensorFlow backend.


In [4]:
cwd = os.getcwd()
print(cwd)
data = '/tf/data'
pickles = '/tf/pickles'
models = '/tf/models'

/tf/smipipeline


In [5]:
# Import modules and config file
configfile = os.path.join(cwd,'config/mri/run_preproc_mri.json')
with open(configfile, "r") as f:
        config = json.load(f)
pp.pprint(config)

{'l3_finder': {'cache_dir': '/tf/_cache/',
               'cache_intermediate_results': True,
               'model_path_dir': '/tf/models/l3/cv_final',
               'new_tim_dicom_dir_structure': True,
               'output_directory': '/tf/output/cv_poorl3/l3',
               'overwrite': True,
               'save_plots': True,
               'show_plots': False,
               'tif_dir': '/tf/data/sarcopeniaMR_L3_TIF'},
 'muscle_segmentor': {'input_directory': '/tf/data/sarcopeniaMR_L3_TIF',
                      'output_directory': '/tf/data/sarcopeniaMR_L3_h5',
                      'sma_config_args_file': '/tf/smipipeline/config/mri/sma_mri_preproc.args'}}


In [6]:
## Preprocess config to convert tif to hdf5 file for training
config = config['muscle_segmentor']
##
config["input_type"] = "Image"
config["input_shape"] = (256,256)
config["input_images"] = config['input_directory']
config["image_format"] = "TIF" # or "NIFTI"
config["slice_number"] = 0 # Use this if you have a stacked TIF and want only one slice for 2D problems.
                           # slice number goes from 0 to length of Stack
config['use_middle_image'] = False

config["output_file"] = "mri.h5"

config["overwrite"] = 1
config["problem_type"] = "Segmentation"
config["image_modalities"] = ["MR"]
config["image_masks"] = ["MASK" ] #["Label"]   # For Image Masks, will serve as label for segmentation problems
config["n_channels"] = 0            # All image channels that will be used as input, image_mask can be input for classification problems and output for segmentation problems.

config["clinical_truthname"] =  None # For CSV File
config["normalize"] = True
config['normalize_on_load'] = True
config["invert_image"] = True

In [7]:
pp.pprint(config)

{'clinical_truthname': None,
 'image_format': 'TIF',
 'image_masks': ['MASK'],
 'image_modalities': ['MR'],
 'input_directory': '/tf/data/sarcopeniaMR_L3_TIF',
 'input_images': '/tf/data/sarcopeniaMR_L3_TIF',
 'input_shape': (256, 256),
 'input_type': 'Image',
 'invert_image': True,
 'n_channels': 0,
 'normalize': True,
 'normalize_on_load': True,
 'output_directory': '/tf/data/sarcopeniaMR_L3_h5',
 'output_file': 'mri.h5',
 'overwrite': 1,
 'problem_type': 'Segmentation',
 'slice_number': 0,
 'sma_config_args_file': '/tf/smipipeline/config/mri/sma_mri_preproc.args',
 'use_middle_image': False}


### Segment L3 Axial Images and Calculate Muscle Area

In [8]:

def get_image_loader(config,problem_type,input_images):
    return mri_TIF_loader(
                problem_type,
                input_images,
                config["input_shape"],
                config["image_modalities"],
                config["image_masks"],
                config['slice_number'],config['invert_image'],config['normalize_on_load'])

In [9]:
# Step 1: Check if Input Folders are defined
try:
    input_type = config["input_type"]
except:
    print("Error: Input type for preprocessing not defined | \t Set  config[\"input_type\"] to \"Image\", \"Clinical\" or \"Both\" \n")
    
input_images = config["input_images"]
input_clinical = None
if (input_type == "Image" or input_type == "Both"):
    try:
        input_images = os.path.abspath(config["input_images"])
    except:
        print("Error: Input Image Folder for preprocessing not defined | \t Set config[\"input_images\"] \n")

if (input_type == "Clinical" or input_type == "Both"):
    try:
        input_clinical = os.path.abspath(config["input_clinical"])
    except:
        print("Error: Input Clinical Folder with .csv for preprocessing not defined | \t Set config[\"input_clinical\"] \n")

In [10]:
# Step 2: Check if the Output File is defined
try:
    output_file = os.path.abspath(os.path.join(config["output_directory"], config["output_file"]))
except:
    print("Error: Input type for preprocessing not defined | \t Set  config[\"input_type\"] to \"Image\",\"Clinical\" or \"Both\" \n")

In [12]:
# Step 3: Check if Output file already exists, If it exists, require user permission to overwrite
if 'overwrite' in config:
    overwrite = config["overwrite"]
elif os.path.exists(output_file):
    overwrite = input("Output file exists, do you want to overwrite? (y/n) \n")
    overwrite = True if overwrite == 'y' else False   
# Open the hdf5 file
hdf5_file = tables.open_file(output_file, mode='w')

In [13]:
# Step 4: Check problem specific parameters are defined
problem_type = config['problem_type']
print('Problem type is: ', problem_type)
print('Input Type is: ', input_type)
print('Input Images is: ', input_images)

Problem type is:  Segmentation
Input Type is:  Image
Input Images is:  /tf/data/sarcopeniaMR_L3_TIF


In [14]:
# Step 5: Convert Exams to hdf5 
#  Load Imaging Data
image_loader = get_image_loader(config,problem_type,input_images)

In [15]:
print(image_loader)

In [16]:
image_storage = image_loader.load_toHDF5(hdf5_file=hdf5_file)

After Resizing image max:  0.6240973874948288
After Resizing image min:  -5.256552577596869
After Resizing image max:  1
After Resizing image min:  0
After Resizing image max:  0.5087909718028527
After Resizing image min:  -5.68411547797939
After Resizing image max:  1
After Resizing image min:  0
After Resizing image max:  0.9186086603012847
After Resizing image min:  -3.9845983964818723
After Resizing image max:  1
After Resizing image min:  0
After Resizing image max:  0.5460244303082225
After Resizing image min:  -6.320902990799771
After Resizing image max:  1
After Resizing image min:  0
After Resizing image max:  0.48719472780774775
After Resizing image min:  -5.521782808038246
After Resizing image max:  1
After Resizing image min:  0
After Resizing image max:  0.6342990306001188
After Resizing image min:  -6.128618515213791
After Resizing image max:  1
After Resizing image min:  0
After Resizing image max:  0.8090762961141962
After Resizing image min:  -4.741822540900883
After R

After Resizing image max:  0.6496741939430422
After Resizing image min:  -6.4206513971743675
After Resizing image max:  1
After Resizing image min:  0
After Resizing image max:  0.8676853629484936
After Resizing image min:  -3.910497248836639
After Resizing image max:  1
After Resizing image min:  0
After Resizing image max:  0.6372212867672148
After Resizing image min:  -5.13265716457957
After Resizing image max:  1
After Resizing image min:  0
After Resizing image max:  0.6887476277638028
After Resizing image min:  -4.8208344636799145
After Resizing image max:  1
After Resizing image min:  0
After Resizing image max:  0.5191040906677546
After Resizing image min:  -6.22594681920116
After Resizing image max:  1
After Resizing image min:  0
After Resizing image max:  0.4321320537090103
After Resizing image min:  -6.098593814160164
After Resizing image max:  1
After Resizing image min:  0
After Resizing image max:  0.6219970671648704
After Resizing image min:  -5.919024880184373
After Re

After Resizing image max:  0.6397202424097678
After Resizing image min:  -5.0941396144338436
After Resizing image max:  1
After Resizing image min:  0
After Resizing image max:  1.0172983797280533
After Resizing image min:  -3.467163885751155
After Resizing image max:  1
After Resizing image min:  0
After Resizing image max:  0.8475481645035359
After Resizing image min:  -4.443874076639503
After Resizing image max:  1
After Resizing image min:  0
After Resizing image max:  0.5698786947315
After Resizing image min:  -6.712277885131613
After Resizing image max:  1
After Resizing image min:  0
After Resizing image max:  0.5375854901659808
After Resizing image min:  -6.517371139839498
After Resizing image max:  1
After Resizing image min:  0
After Resizing image max:  0.6852003026936859
After Resizing image min:  -4.1155856214486395
After Resizing image max:  1
After Resizing image min:  0
After Resizing image max:  0.7052344424886878
After Resizing image min:  -4.853634025376292
After Res

After Resizing image max:  0.8147035309194445
After Resizing image min:  -3.540385447468592
After Resizing image max:  1
After Resizing image min:  0
After Resizing image max:  0.6720571763040369
After Resizing image min:  -5.35774567157958
After Resizing image max:  1
After Resizing image min:  0
After Resizing image max:  0.6668433462430389
After Resizing image min:  -4.761479310106166
After Resizing image max:  1
After Resizing image min:  0
After Resizing image max:  0.514926592383317
After Resizing image min:  -6.682453253245665
After Resizing image max:  1
After Resizing image min:  0
After Resizing image max:  0.49122111220702913
After Resizing image min:  -6.586735420229533
After Resizing image max:  1
After Resizing image min:  0
After Resizing image max:  0.6995015203785905
After Resizing image min:  -5.494362003758942
After Resizing image max:  1
After Resizing image min:  0
After Resizing image max:  0.5963873948591355
After Resizing image min:  -4.5344633622016195
After Re

In [17]:
subject_ids_final = image_loader.get_sample_ids()
id_storage = hdf5_file.create_array(hdf5_file.root, 'subject_ids', obj=subject_ids_final)
print("Input Data Preprocessed and Loaded to HDF5")

Input Data Preprocessed and Loaded to HDF5


In [11]:
#debug
#hdf5_file = tables.open_file(output_file, mode='r+')
#image_storage = hdf5_file.root.imdata

In [15]:
# Step 7: Normalize Data Storage
if config["normalize"]:
    if len(config["input_shape"]) > 2:
        normalize_data_storage(image_storage)
        print("Data in HDF5 File is normalized for training")    
    else:
        mean,std = normalize_data_storage_2D(image_storage)
        imdata_mean_std = hdf5_file.create_array(hdf5_file.root, 'mean_std', obj=[mean,std])
        print("Data in HDF5 File is normalized for training")  

mean: [-9.489297e-08] std: [0.99999994]
Data in HDF5 File is normalized for training


In [28]:
hdf5_file.close()